# Train a model with PyTorch

> This notebook is a simplified version of our workflow. It exposes the basic details of the traning and evaluation loop more explicitly, but does not offer advanced features like early stopping, mini-batches or validation. Use the `*-lightning` version for those.

## How to use

Run `python run_notebook.py --help` for more information.

In [ ]:
# If this is the template file (and not a copy) and you are introducing changes,
# update VERSION with the current date (YYYY.MM.DD)
VERSION = "2021.04.09"

## ✏ Define hyper parameters

In [ ]:
# TEMPLATE VALUES -- these are overriden (see below if executed) by papermill using a YAML or Python file as input

# DATA -- Glob paths must be relative to the root of the repository: REPO / features
NPZ_FILES = [
    "path/to/*.npz",
]

# Model -- specified with the full import path to the class object
MODEL_CLS = "kinoml.ml.torch_models.NeuralNetworkRegression"
MODEL_KWARGS = {"hidden_size": 350}  # input_shape is defined dynamically during training
WITH_OBSERVATION_MODEL = True

# Adam
LEARNING_RATE = 0.001
EPSILON = 1e-7
BETAS = 0.9, 0.999

# Trainer
MAX_EPOCHS = 50
N_SPLITS = 5
SHUFFLE_FOLDS = False
VALIDATION = False  # TODO: VALIDATION=True is not implemented yet!
MIN_ITEMS_PER_DATASET = 50  # skip datasets if len(data) < N

# Bootstrapping
N_BOOTSTRAPS = 1
BOOTSTRAP_SAMPLE_RATIO = 1

# Output
VERBOSE = False

## IGNORE THIS ONE
HERE = _dh[-1]

⚠ From here on, you should _not_ need to modify anything else 🤞

---

Define key paths for data and outputs:

In [ ]:
from pathlib import Path
from datetime import datetime

HERE = Path(HERE)

for parent in HERE.parents:
    if next(parent.glob(".github/"), None):
        REPO = parent
        break

FEATURES_STORE = REPO / "features"
        
OUT = HERE / "_output" / datetime.now().strftime("%Y%m%d-%H%M%S")
OUT.mkdir(parents=True, exist_ok=True)

print(f"This notebook:           HERE = ~/{HERE.relative_to(Path.home())}")
print(f"This repo:               REPO = ~/{REPO.relative_to(Path.home())}")
print(f"Features:      FEATURES_STORE = ~/{FEATURES_STORE.relative_to(Path.home())}")
print(f"Outputs in:               OUT = ~/{OUT.relative_to(Path.home())}")

In [ ]:
# Nasty trick: save all-caps local variables (CONSTANTS working as hyperparameters) so far in a dict to save it later
_hparams = {key: value for key, value in locals().items() if key.upper() == key and not key.startswith(("_", "OE_"))}

In [ ]:
# TODO: Make all datasets use the same kinase identifiers
ONE_KINASE = {
    "ChEMBLDatasetProvider": "P35968",
    "PKIS2DatasetProvider": "ABL2",
}

In [ ]:
from collections import defaultdict
from warnings import warn
import sys
import shutil

from IPython.display import Markdown
import numpy as np
import pandas as pd
import torch
from torch.utils.data import DataLoader, SubsetRandomSampler
import pytorch_lightning as pl

from kinoml.utils import seed_everything, import_object
from kinoml.core import measurements as measurement_types
from kinoml.datasets.torch_datasets import XyNpzTorchDataset
from kinoml.core.measurements import null_observation_model

# Fix the seed for reproducible random splits -- otherwise we get mixed train/test groups every time, biasing the model evaluation
seed_everything();
print("Run started at", datetime.now())

## Load featurized data and create observation models

We assume this path structure: `$REPO/features/_output/<FEATURIZATION>/<DATASET>/<GROUP>.npz`

In [ ]:
DATASETS = []
MEASUREMENT_TYPES = set()
KINASES = set()
FEATURIZATIONS = set()
for glob in NPZ_FILES:
    npzs = list(FEATURES_STORE.glob(glob))
    if not npzs:
        warn(f"⚠ NPZ glob `{glob}` did not match any files!")
        continue
        
    for npz in npzs:
        kinase, measurement_type = npz.stem.split("__")
        dataset = npz.parent.name
        featurization = npz.parents[1].name
        
        MEASUREMENT_TYPES.add(measurement_type)
        KINASES.add(kinase)
        FEATURIZATIONS.add(featurization)
        
        ds = XyNpzTorchDataset(npz)
        ds.metadata = {
            "kinase": kinase,
            "measurement_type": measurement_type,
            "dataset": dataset,
            "featurization": featurization
        }
        DATASETS.append(ds)
        if not VALIDATION:
            ds.indices["test"] = np.concatenate([ds.indices["test"], ds.indices["val"]])
            ds.indices["val"] = np.array([])

if not DATASETS:
    raise ValueError("Provided `NPZ_FILES` did not result in any valid datasets!")

In [ ]:
print("Observed...")
print(" - Measurement types:", len(MEASUREMENT_TYPES), "-->", *MEASUREMENT_TYPES)
print(" - Kinases:", len(KINASES), "-->", *KINASES)

## Check X duplication

There's a chance we have several measurements per ligand, or, depending on the featurization scheme, even hash collisions... Let's quantify the amount of input tensor duplication we are facing.

In [ ]:
for mtype in MEASUREMENT_TYPES:
    display(Markdown(f"#### {mtype}"))
    unique = {}
    for ds in DATASETS:
        if ds.metadata["measurement_type"] == mtype:
            all_ = ds.data_X.shape[0]
            unique_ = np.unique(ds.data_X, axis=0).shape[0]
            unique[ds.metadata["kinase"]] = {"all": all_, "unique": unique_}
    df = pd.DataFrame.from_dict(unique).T
    df["uniqueness"] = df["unique"] / df["all"]
    # This is how you highlight rows in pandas!
    df = df.describe().style.apply(lambda x: ['font-weight: bold' for v in x], subset=pd.IndexSlice[["mean", "std"], :])
    display(df)

Now that we have all the data-dependent objects, we can start with the model-specific definitions.

### Training loop

In [ ]:
from kinoml.ml.lightning_modules import KFold3Way, KFold
from IPython.display import Markdown
from tqdm.auto import trange, tqdm
from kinoml.ml.torch_models import NeuralNetworkRegression
from ipywidgets import HBox, VBox, Output, HTML
from kinoml.analysis.plots import predicted_vs_observed, performance
from kinoml.utils import fill_until_next_multiple
import pandas as pd
import torch.nn as nn

if VALIDATION:
    kfold = KFold3Way(n_splits=N_SPLITS, shuffle=SHUFFLE_FOLDS)
    ttypes = ["train", "val", "test"]
else:
    kfold = KFold(n_splits=N_SPLITS, shuffle=SHUFFLE_FOLDS)
    ttypes = ["train", "test"]

ModelCls = import_object(MODEL_CLS)
    
kinase_metrics = defaultdict(dict)
for dataset in tqdm(DATASETS):
    kinase = dataset.metadata["kinase"]
    mtype = dataset.metadata["measurement_type"]
    if dataset.data_X.shape[0] < MIN_ITEMS_PER_DATASET:
        warn(f"Ignoring {kinase} because it has less than {MIN_ITEMS_PER_DATASET} entries for type {mtype}")
        continue
            
    if VERBOSE:
        display(Markdown(f"#### {mtype}"))

    mtype_class = getattr(measurement_types, mtype)
    obs_model = mtype_class.observation_model(backend="pytorch")
    metrics = defaultdict(list)

    for fold_index, splits in enumerate(kfold.split(dataset.data_X, dataset.data_y)):
        if VALIDATION:
            train_indices, val_indices, test_indices = splits
        else:
            train_indices, test_indices = splits

        if VERBOSE:
            display(Markdown(f"##### Fold {fold_index}"))

        ####
        # TRAIN
        ####
        x_train = dataset.data_X[train_indices].float()
        x_test = dataset.data_X[test_indices].float()
        y_train = dataset.data_y[train_indices]
        y_test = dataset.data_y[test_indices]

        if VALIDATION:
            x_val = dataset.data_X[val_indices].float()
            y_val = dataset.data_y[val_indices]
        
        input_shape = ModelCls.estimate_input_shape(x_train)
        nn_model = ModelCls(input_shape=input_shape, **MODEL_KWARGS)
        nn_model.train(True)

        optimizer = torch.optim.Adam(nn_model.parameters(), lr=LEARNING_RATE, eps=EPSILON, betas=BETAS)
        loss_function = torch.nn.MSELoss()

        if VERBOSE:
            range_epochs = trange(MAX_EPOCHS, desc="Epochs (+ featurization...)")
        else:
            range_epochs = range(MAX_EPOCHS)
        for epoch in range_epochs:
            optimizer.zero_grad()

            prediction = nn_model(x_train)
            if WITH_OBSERVATION_MODEL:
                prediction = obs_model(prediction)

            prediction = prediction.view_as(y_train)

            loss = loss_function(prediction, y_train)
            if VERBOSE:
                range_epochs.set_description(f"Epochs (loss={loss.item():.2e})")

            if VALIDATION:
                warn("Validation step not implemented yet")


            # Gradients w.r.t. parameters
            loss.backward()

            # Optimizer
            optimizer.step()
        
        ###
        # Save model's state -- you will still need to instantiate the model class!
        # Possibly using something like:
        # model = import_object(MODEL_CLS)(**MODEL_KWARGS)
        # model.load_state_dict(torch.load("state_dict.pt"))
        ###
        torch.save(nn_model.state_dict(), OUT / f"state_dict_{kinase}_{mtype}_fold{fold_index}.pt")
        
        ####
        # EVAL
        ####
        nn_model.eval()
        outputs = []
        for ttype in ttypes:
            output = Output()
            with output:
                title = f"fold={fold_index}, {ttype}={locals()[f'{ttype}_indices'].shape[0]}"
                print(title)
                print("-"*(len(title)))

                observed = locals()[f"y_{ttype}"]

                with torch.no_grad():
                    predicted = nn_model(locals()[f"x_{ttype}"])
                    if WITH_OBSERVATION_MODEL:
                        predicted = obs_model(predicted)

                predicted = predicted.view_as(observed).detach().numpy()
                observed = observed.detach().numpy()
                these_metrics = performance(predicted, observed, n_boot=N_BOOTSTRAPS, sample_ratio=BOOTSTRAP_SAMPLE_RATIO)
                metrics[ttype].append(these_metrics)
                if VERBOSE:
                    display(predicted_vs_observed(predicted, observed, mtype_class, with_metrics=False))

            outputs.append(output)
        if VERBOSE:
            display(HBox(outputs))

    # Average performances

    average = defaultdict(dict)
    for key in metrics["test"][0]:
        for label in ttypes:
            # this zero here ---v is super important! we only want the mean of the means!
            values =  [fold[key][0] for fold in metrics[label]]
            average[label][key] = {
                "mean": np.mean(values),
                "std": np.std(values)
            }
    if VERBOSE:
        for label in ttypes:    
            display(HTML(f"Bootstrapped average across folds ({label}):"))
            display(pd.DataFrame.from_dict(average[label]))
    kinase_metrics[kinase][mtype] = average

### Summary

`kinase_metrics` is a nested dictionary with these dimensions:

- kinase name
- measurement type
- metric
- mean & standard deviation

In [ ]:
import json

display(Markdown(f"""
### Configuration 

```json
{json.dumps(_hparams, default=str, indent=2)}
```
"""))

if VERBOSE:
    display(Markdown(f"""

    ### Kinase metrics

    ```json
    {json.dumps(kinase_metrics, default=str, indent=2)}
    ```
    """))

In [ ]:
for mtype in MEASUREMENT_TYPES:
    display(Markdown(f"#### {mtype}"))

    dict_of_flattened_metrics = {}
    for kinase_name, measurement_type_dict in sorted(kinase_metrics.items(), key=lambda kv: kv[0].lower()):
        flattened_metrics = {}
        for train_test_key, train_test_dict in measurement_type_dict.get(mtype, {}).items():
            for metric_key, mean_std_dict in train_test_dict.items():
                for mean_std_key, value in mean_std_dict.items():
                    flattened_metrics[f"{train_test_key}_{metric_key}_{mean_std_key}"] = (value,)
        if flattened_metrics:
            dict_of_flattened_metrics[kinase_name] = pd.DataFrame.from_dict(flattened_metrics)
    
    if not dict_of_flattened_metrics:
        continue
    
    df = pd.concat(dict_of_flattened_metrics)
    df.index = [index[0] for index in df.index]
    with pd.option_context("display.float_format", "{:.3f}".format, "display.max_rows", len(df)):
        display(df.style.background_gradient(subset=["train_r2_mean", "test_r2_mean"], low=0, high=1, vmin=0, vmax=1))
        display(df.describe()[["train_r2_mean", "train_r2_std", "test_r2_mean", "test_r2_std"]].describe().style.apply(lambda x: ['font-weight: bold' for v in x], subset=pd.IndexSlice[["mean", "std"], :]))

In [ ]:
print("Run finished at", datetime.now())

### Save reports to disk

In [ ]:
from kinoml.utils import watermark
w = watermark()

In [ ]:
%%capture cap --no-stderr
w = watermark()

In [ ]:
import json

df.to_csv(OUT / "performance.csv")

with open(OUT / "performance.json", "w") as f:
    json.dump(kinase_metrics, f, default=str, indent=2)
    
with open(OUT/ "watermark.txt", "w") as f:
    f.write(cap.stdout)

with open(OUT / "hparams.json", "w") as f:
    json.dump(_hparams, f, default=str, indent=2)